#**Cycle GAN**

Playing Master



### Dependencies

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import numpy as np
import scipy.misc

from functools import partial

import tensorflow as tf
import tensorflow.contrib.slim as slim

import copy
import os
import re


import argparse
from glob import glob

### Model (Generator + Discriminator)

In [0]:
conv = partial(slim.conv2d, activation_fn=None)
deconv = partial(slim.conv2d_transpose, activation_fn=None)
relu = tf.nn.relu
lrelu = partial(tf.nn.leaky_relu, alpha=0.2)
batch_norm = partial(slim.batch_norm, scale=True, decay=0.9, epsilon=1e-5, updates_collections=None)


def discriminator(img, scope, dim=64, train=True):
    bn = partial(batch_norm, is_training=train)
    conv_bn_lrelu = partial(conv, normalizer_fn=bn, activation_fn=lrelu, biases_initializer=None)

    with tf.variable_scope(scope + '_discriminator', reuse=tf.AUTO_REUSE):
        net = lrelu(conv(img, dim, 4, 2))
        net = conv_bn_lrelu(net, dim * 2, 4, 2)
        net = conv_bn_lrelu(net, dim * 4, 4, 2)
        net = conv_bn_lrelu(net, dim * 8, 4, 1)
        net = conv(net, 1, 4, 1)

        return net


def generator(img, scope, dim=64, train=True):
    bn = partial(batch_norm, is_training=train)
    conv_bn_relu = partial(conv, normalizer_fn=bn, activation_fn=relu, biases_initializer=None)
    deconv_bn_relu = partial(deconv, normalizer_fn=bn, activation_fn=relu, biases_initializer=None)

    def _residule_block(x, dim):
        y = conv_bn_relu(x, dim, 3, 1)
        y = bn(conv(y, dim, 3, 1))
        return y + x

    with tf.variable_scope(scope + '_generator', reuse=tf.AUTO_REUSE):
        net = conv_bn_relu(img, dim, 7, 1)
        net = conv_bn_relu(net, dim * 2, 3, 2)
        net = conv_bn_relu(net, dim * 4, 3, 2)

        for i in range(9):
            net = _residule_block(net, dim * 4)

        net = deconv_bn_relu(net, dim * 2, 3, 2)
        net = deconv_bn_relu(net, dim, 3, 2)
        net = conv(net, 3, 7, 1)
        net = tf.nn.tanh(net)

        return net

### Helper Functions

In [0]:
def counter(scope='counter'):
    with tf.variable_scope(scope):
        counter = tf.Variable(0, dtype=tf.int32, name='counter')
        update_cnt = tf.assign(counter, tf.add(counter, 1))
        return counter, update_cnt


def summary(tensor_collection,
            summary_type=['mean', 'stddev', 'max', 'min', 'sparsity', 'histogram'],
            scope=None):
    """Summary.

    usage:
        1. summary(tensor)
        2. summary([tensor_a, tensor_b])
        3. summary({tensor_a: 'a', tensor_b: 'b})
    """
    def _summary(tensor, name, summary_type):
        """Attach a lot of summaries to a Tensor."""
        if name is None:
            # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
            # session. This helps the clarity of presentation on tensorboard.
            name = re.sub('%s_[0-9]*/' % 'tower', '', tensor.name)
            name = re.sub(':', '-', name)

        summaries = []
        if len(tensor.shape) == 0:
            summaries.append(tf.summary.scalar(name, tensor))
        else:
            if 'mean' in summary_type:
                mean = tf.reduce_mean(tensor)
                summaries.append(tf.summary.scalar(name + '/mean', mean))
            if 'stddev' in summary_type:
                mean = tf.reduce_mean(tensor)
                stddev = tf.sqrt(tf.reduce_mean(tf.square(tensor - mean)))
                summaries.append(tf.summary.scalar(name + '/stddev', stddev))
            if 'max' in summary_type:
                summaries.append(tf.summary.scalar(name + '/max', tf.reduce_max(tensor)))
            if 'min' in summary_type:
                summaries.append(tf.summary.scalar(name + '/min', tf.reduce_min(tensor)))
            if 'sparsity' in summary_type:
                summaries.append(tf.summary.scalar(name + '/sparsity', tf.nn.zero_fraction(tensor)))
            if 'histogram' in summary_type:
                summaries.append(tf.summary.histogram(name, tensor))
        return tf.summary.merge(summaries)

    if not isinstance(tensor_collection, (list, tuple, dict)):
        tensor_collection = [tensor_collection]

    with tf.name_scope(scope, 'summary'):
        summaries = []
        if isinstance(tensor_collection, (list, tuple)):
            for tensor in tensor_collection:
                summaries.append(_summary(tensor, None, summary_type))
        else:
            for tensor, name in tensor_collection.items():
                summaries.append(_summary(tensor, name, summary_type))
        return tf.summary.merge(summaries)


def load_checkpoint(ckpt_dir_or_file, session, var_list=None):
    """Load checkpoint.

    Note:
        This function add some useless ops to the graph. It is better
        to use tf.train.init_from_checkpoint(...).
    """
    print(' [*] Loading checkpoint...')
    if os.path.isdir(ckpt_dir_or_file):
        ckpt_dir_or_file = tf.train.latest_checkpoint(ckpt_dir_or_file)

    restorer = tf.train.Saver(var_list)
    restorer.restore(session, ckpt_dir_or_file)
    print(' [*] Loading succeeds! Copy variables from % s' % ckpt_dir_or_file)


class ItemPool(object):

    def __init__(self, max_num=50):
        self.max_num = max_num
        self.num = 0
        self.items = []

    def __call__(self, in_items):
        # in_items is a list of item
        if self.max_num == 0:
            return in_items
        return_items = []
        for in_item in in_items:
            if self.num < self.max_num:
                self.items.append(in_item)
                self.num = self.num + 1
                return_items.append(in_item)
            else:
                if np.random.rand() > 0.5:
                    idx = np.random.randint(0, self.max_num)
                    tmp = copy.copy(self.items[idx])
                    self.items[idx] = in_item
                    return_items.append(tmp)
                else:
                    return_items.append(in_item)
        return return_items


def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path)


In [0]:
def _to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    # transform images from [-1.0, 1.0] to [min_value, max_value] of dtype
    assert \
        np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        'The input images should be float64(32) and in the range of [-1.0, 1.0]!'
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) + min_value).astype(dtype)


def _im2uint(images):
    # transform images from [-1.0, 1.0] to uint8
    return _to_range(images, 0, 255, np.uint8)


def imread(path, mode='RGB'):
    """Read an image.

    read an image into [-1.0, 1.0] of float64

    `mode` can be one of the following strings:

    * 'L' (8 - bit pixels, black and white)
    * 'P' (8 - bit pixels, mapped to any other mode using a color palette)
    * 'RGB' (3x8 - bit pixels, true color)
    * 'RGBA' (4x8 - bit pixels, true color with transparency mask)
    * 'CMYK' (4x8 - bit pixels, color separation)
    * 'YCbCr' (3x8 - bit pixels, color video format)
    * 'I' (32 - bit signed integer pixels)
    * 'F' (32 - bit floating point pixels)
    """
    return scipy.misc.imread(path, mode=mode) / 127.5 - 1


def imwrite(image, path):
    # save an [-1.0, 1.0] image
    return scipy.misc.imsave(path, _to_range(image, 0, 255, np.uint8))


def imresize(image, size, interp='bilinear'):
    """Resize an image.

    Resize an [-1.0, 1.0] image.

    size: int, float or tuple
        * int - Percentage of current size.
        * float - Fraction of current size.
        * tuple - Size of the output image.

    interp: str, optional
        Interpolation to use for re - sizing('nearest', 'lanczos', 'bilinear', 'bicubic'
        or 'cubic').
    """
    # scipy.misc.imresize should deal with uint8 image, or it would cause some problem (scale the image to [0, 255])
    return (scipy.misc.imresize(_im2uint(image), size, interp=interp) / 127.5 - 1).astype(image.dtype)


def immerge(images, row, col):
    """Merge images.

    merge images into an image with (row * h) * (col * w)

    `images` is in shape of N * H * W(* C=1 or 3)
    """
    if images.ndim == 4:
        c = images.shape[3]
    elif images.ndim == 3:
        c = 1

    h, w = images.shape[1], images.shape[2]
    if c > 1:
        img = np.zeros((h * row, w * col, c))
    else:
        img = np.zeros((h * row, w * col))
    for idx, image in enumerate(images):
        i = idx % col
        j = idx // col
        img[j * h:j * h + h, i * w:i * w + w, ...] = image

    return img


In [0]:
class ImageData:

    def __init__(self,
                 session,
                 image_paths,
                 batch_size,
                 load_size=286,
                 crop_size=256,
                 channels=3,
                 prefetch_batch=2,
                 drop_remainder=True,
                 num_threads=16,
                 shuffle=True,
                 buffer_size=4096,
                 repeat=-1):

        self._sess = session
        self._img_batch = ImageData._image_batch(image_paths,
                                                 batch_size,
                                                 load_size,
                                                 crop_size,
                                                 channels,
                                                 prefetch_batch,
                                                 drop_remainder,
                                                 num_threads,
                                                 shuffle,
                                                 buffer_size,
                                                 repeat)
        self._img_num = len(image_paths)

    def __len__(self):
        return self._img_num

    def batch(self):
        return self._sess.run(self._img_batch)

    @staticmethod
    def _image_batch(image_paths,
                     batch_size,
                     load_size=286,
                     crop_size=256,
                     channels=3,
                     prefetch_batch=2,
                     drop_remainder=True,
                     num_threads=16,
                     shuffle=True,
                     buffer_size=4096,
                     repeat=-1):
        def _parse_func(path):
            img = tf.read_file(path)
            img = tf.image.decode_jpeg(img, channels=channels)
            img = tf.image.random_flip_left_right(img)
            img = tf.image.resize_images(img, [load_size, load_size])
            img = (img - tf.reduce_min(img)) / (tf.reduce_max(img) - tf.reduce_min(img))
            img = tf.random_crop(img, [crop_size, crop_size, channels])
            img = img * 2 - 1
            return img

        dataset = tf.data.Dataset.from_tensor_slices(image_paths)

        dataset = dataset.map(_parse_func, num_parallel_calls=num_threads)

        if shuffle:
            dataset = dataset.shuffle(buffer_size)

        if drop_remainder:
            dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
        else:
            dataset = dataset.batch(batch_size)

        dataset = dataset.repeat(repeat).prefetch(prefetch_batch)

        return dataset.make_one_shot_iterator().get_next()


### Training

In [27]:
tf.reset_default_graph()
""" param """
dataset = 'horse2zebra'
load_size = 286
crop_size = 256
epoch = 2
batch_size = 1
lr = 0.0002
clip = 0.01


""" graph """
# models
generator_a2b = partial(generator, scope='a2b')
generator_b2a = partial(generator, scope='b2a')
discriminator_a = partial(discriminator, scope='a')
discriminator_b = partial(discriminator, scope='b')

# operations

a_real = tf.placeholder(tf.float32, shape=[None, crop_size, crop_size, 3])
b_real = tf.placeholder(tf.float32, shape=[None, crop_size, crop_size, 3])
a2b_sample = tf.placeholder(tf.float32, shape=[None, crop_size, crop_size, 3])
b2a_sample = tf.placeholder(tf.float32, shape=[None, crop_size, crop_size, 3])

a2b = generator_a2b(a_real)
b2a = generator_b2a(b_real)
b2a2b = generator_a2b(b2a)
a2b2a = generator_b2a(a2b)

a_logit = discriminator_a(a_real)
b2a_logit = discriminator_a(b2a)
b2a_sample_logit = discriminator_a(b2a_sample)
b_logit = discriminator_b(b_real)
a2b_logit = discriminator_b(a2b)
a2b_sample_logit = discriminator_b(a2b_sample)

# losses
# g_loss_a2b = tf.losses.mean_squared_error(a2b_logit, tf.ones_like(a2b_logit))
# g_loss_b2a = tf.losses.mean_squared_error(b2a_logit, tf.ones_like(b2a_logit))
# cyc_loss_a = tf.losses.absolute_difference(a_real, a2b2a)
# cyc_loss_b = tf.losses.absolute_difference(b_real, b2a2b)
# g_loss = g_loss_a2b + g_loss_b2a + cyc_loss_a * 10.0 + cyc_loss_b * 10.0

# d_loss_a_real = tf.losses.mean_squared_error(a_logit, tf.ones_like(a_logit))
# d_loss_b2a_sample = tf.losses.mean_squared_error(b2a_sample_logit, tf.zeros_like(b2a_sample_logit))
# d_loss_a = d_loss_a_real + d_loss_b2a_sample

# d_loss_b_real = tf.losses.mean_squared_error(b_logit, tf.ones_like(b_logit))
# d_loss_a2b_sample = tf.losses.mean_squared_error(a2b_sample_logit, tf.zeros_like(a2b_sample_logit))
# d_loss_b = d_loss_b_real + d_loss_a2b_sample

# WGAN losses
g_loss_a2b = tf.losses.mean_squared_error(a2b_logit, tf.ones_like(a2b_logit))
g_loss_b2a = tf.losses.mean_squared_error(b2a_logit, tf.ones_like(b2a_logit))
cyc_loss_a = tf.losses.absolute_difference(a_real, a2b2a)
cyc_loss_b = tf.losses.absolute_difference(b_real, b2a2b)
g_loss = g_loss_a2b + g_loss_b2a + cyc_loss_a * 10.0 + cyc_loss_b * 10.0

wd_a = tf.reduce_mean(a_logit) - tf.reduce_mean(a2b_logit)
d_loss_a = -wd_a

wd_b = tf.reduce_mean(b_logit) - tf.reduce_mean(b2a_logit)
d_loss_b = -wd_b


# summaries
g_summary = summary({g_loss_a2b: 'g_loss_a2b',
                           g_loss_b2a: 'g_loss_b2a',
                           cyc_loss_a: 'cyc_loss_a',
                           cyc_loss_b: 'cyc_loss_b'})
d_summary_a = summary({d_loss_a: 'd_loss_a'})
d_summary_b = summary({d_loss_b: 'd_loss_b'})

# optim
t_var = tf.trainable_variables()
d_a_var = [var for var in t_var if 'a_discriminator' in var.name]
d_b_var = [var for var in t_var if 'b_discriminator' in var.name]
g_var = [var for var in t_var if 'a2b_generator' in var.name or 'b2a_generator' in var.name]

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    d_a_train_op = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(d_loss_a, var_list=d_a_var)
    with tf.control_dependencies([d_a_train_op]):
        d_a_train_op = tf.group(*(tf.assign(var, tf.clip_by_value(var, -clip, clip)) for var in d_a_var))
    d_b_train_op = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(d_loss_b, var_list=d_b_var)
    with tf.control_dependencies([d_b_train_op]):
        d_b_train_op = tf.group(*(tf.assign(var, tf.clip_by_value(var, -clip, clip)) for var in d_b_var))
    g_train_op = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(g_loss, var_list=g_var)


""" train """
''' init '''
# session
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# counter
it_cnt, update_cnt = counter()

''' data '''

a_img_paths = glob('/content/drive/My Drive/Colab Notebooks/datasets/testA/*.jpg')
b_img_paths = glob('/content/drive/My Drive/Colab Notebooks/datasets/testB/*.jpg')
a_data_pool = ImageData(sess, a_img_paths, batch_size, load_size=load_size, crop_size=crop_size)
b_data_pool = ImageData(sess, b_img_paths, batch_size, load_size=load_size, crop_size=crop_size)

a_test_img_paths = glob('/content/drive/My Drive/Colab Notebooks/datasets/testA/*.jpg')
b_test_img_paths = glob('/content/drive/My Drive/Colab Notebooks/datasets/testB/*.jpg')
a_test_pool = ImageData(sess, a_test_img_paths, batch_size, load_size=load_size, crop_size=crop_size)
b_test_pool = ImageData(sess, b_test_img_paths, batch_size, load_size=load_size, crop_size=crop_size)

a2b_pool = ItemPool()
b2a_pool = ItemPool()

''' summary '''
summary_writer = tf.summary.FileWriter('/content/drive/My Drive/Colab Notebooks/outputs/summaries', sess.graph)

''' saver '''
saver = tf.train.Saver(max_to_keep=5)

''' restore '''
ckpt_dir = '/content/drive/My Drive/Colab Notebooks/checkpoints'
mkdir(ckpt_dir)
try:
    load_checkpoint(ckpt_dir, sess)
except:
    sess.run(tf.global_variables_initializer())

'''train'''
try:
    batch_epoch = min(len(a_data_pool), len(b_data_pool)) // batch_size
    max_it = epoch * batch_epoch
#     max_it = 300
    for it in range(sess.run(it_cnt), max_it):
        sess.run(update_cnt)
        epoch = it // batch_epoch
        it_epoch = it % batch_epoch + 1

        # prepare data
        a_real_ipt = a_data_pool.batch()
        b_real_ipt = b_data_pool.batch()
        a2b_opt, b2a_opt = sess.run([a2b, b2a], feed_dict={a_real: a_real_ipt, b_real: b_real_ipt})
        a2b_sample_ipt = np.array(a2b_pool(list(a2b_opt)))
        b2a_sample_ipt = np.array(b2a_pool(list(b2a_opt)))

        # train G
        g_summary_opt, _ = sess.run([g_summary, g_train_op], feed_dict={a_real: a_real_ipt, b_real: b_real_ipt})
        summary_writer.add_summary(g_summary_opt, it)
        # train D_b
        d_summary_b_opt, _ = sess.run([d_summary_b, d_b_train_op], feed_dict=   {b_real: b_real_ipt, a2b_sample: a2b_sample_ipt})
        summary_writer.add_summary(d_summary_b_opt, it)
        # train D_a
        d_summary_a_opt, _ = sess.run([d_summary_a, d_a_train_op], feed_dict={a_real: a_real_ipt, b2a_sample: b2a_sample_ipt})
        summary_writer.add_summary(d_summary_a_opt, it)

        # display
        if it % 119 == 0:
            print("Epoch: (%3d) (%5d/%5d)" % (epoch, it_epoch, batch_epoch))

        # save
        if (it + 1) % 119 == 0:
            save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
            print('Model saved in file: % s' % save_path)

        # sample
        if (it + 1) % 119 == 0:
            a_real_ipt = a_test_pool.batch()
            b_real_ipt = b_test_pool.batch()
            [a2b_opt, a2b2a_opt, b2a_opt, b2a2b_opt] = sess.run([a2b, a2b2a, b2a, b2a2b], feed_dict={a_real: a_real_ipt, b_real: b_real_ipt})
            sample_opt = np.concatenate((a_real_ipt, a2b_opt, a2b2a_opt, b_real_ipt, b2a_opt, b2a2b_opt), axis=0)
                
            save_dir = '/content/drive/My Drive/Colab Notebooks/outputs/sample_images_while_training'
            mkdir(save_dir)
            imwrite(immerge(sample_opt, 2, 3), '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, it_epoch, batch_epoch))
except:
    save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
    print('Model saved in file: % s' % save_path)
    sess.close()


 [*] Loading checkpoint...
Epoch: (  0) (    1/  120)
Model saved in file: /content/drive/My Drive/Colab Notebooks/checkpoints/Epoch_(0)_(119of120).ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Epoch: (  0) (  120/  120)
Model saved in file: /content/drive/My Drive/Colab Notebooks/checkpoints/Epoch_(1)_(118of120).ckpt
Epoch: (  1) (  119/  120)


### Testing

In [0]:
tf.reset_default_graph()
""" param """
dataset = 'horse2zebra'
crop_size = 256


""" run """
with tf.Session() as sess:
    a_real = tf.placeholder(tf.float32, shape=[None, crop_size, crop_size, 3])
    b_real = tf.placeholder(tf.float32, shape=[None, crop_size, crop_size, 3])

    a2b = generator(a_real, 'a2b')
    b2a = generator(b_real, 'b2a')
    b2a2b = generator(b2a, 'a2b')
    a2b2a = generator(a2b, 'b2a')

    # retore
    try:
        ckpt_path = load_checkpoint('/content/drive/My Drive/Colab Notebooks/checkpoints', sess)
    except:
        raise Exception('No checkpoint!')

    # test
    a_list = glob('/content/drive/My Drive/Colab Notebooks/datasets/testA/*.jpg')
    b_list = glob('/content/drive/My Drive/Colab Notebooks/datasets/testB/*.jpg')

    a_save_dir = '/content/drive/My Drive/Colab Notebooks/outputs/test_predictions/testA'
    b_save_dir = '/content/drive/My Drive/Colab Notebooks/outputs/test_predictions/testB'
    mkdir([a_save_dir, b_save_dir])

    for i in range(len(a_list)):
        a_real_ipt = imresize(imread(a_list[i]), [crop_size, crop_size])
        a_real_ipt.shape = 1, crop_size, crop_size, 3
        a2b_opt, a2b2a_opt = sess.run([a2b, a2b2a], feed_dict={a_real: a_real_ipt})
        a_img_opt = np.concatenate((a_real_ipt, a2b_opt, a2b2a_opt), axis=0)

        img_name = os.path.basename(a_list[i])
        imwrite(immerge(a_img_opt, 1, 3), a_save_dir + '/' + img_name)
        print('Save %s' % (a_save_dir + '/' + img_name))

    for i in range(len(b_list)):
        b_real_ipt = imresize(imread(b_list[i]), [crop_size, crop_size])
        b_real_ipt.shape = 1, crop_size, crop_size, 3
        b2a_opt, b2a2b_opt = sess.run([b2a, b2a2b], feed_dict={b_real: b_real_ipt})
        b_img_opt = np.concatenate((b_real_ipt, b2a_opt, b2a2b_opt), axis=0)

        img_name = os.path.basename(b_list[i])
        imwrite(immerge(b_img_opt, 1, 3), b_save_dir + '/' + img_name)
        print('Save %s' % (b_save_dir + '/' + img_name))


 [*] Loading checkpoint...
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/checkpoints/Epoch_(99)_(150of150).ckpt
 [*] Loading succeeds! Copy variables from /content/drive/My Drive/Colab Notebooks/checkpoints/Epoch_(99)_(150of150).ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_22439.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_5250.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_3391.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_1155.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_6405.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_10661.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_4004.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_1244.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(256)/n02085936_11653.jpg
Save /content/drive/My Drive/Colab Notebooks/outputs/test_predictions/maltese(2